# HW1--Group 6 

kaidx@kth.se & zhenlinz@kth.se

Task

You are to implement the stages of finding textually similar documents based on Jaccard similarity using the shingling, minhashing, and locality-sensitive hashing (LSH) techniques and corresponding algorithms. The implementation can be done using any big data processing framework, such as Apache Spark, Apache Flink, or no framework, e.g., in Java, Python, etc. To test and evaluate your implementation, write a program that uses your implementation to find similar documents in a corpus of 5-10 or more documents such as web pages or emails.

The stages should be implemented as a collection of classes, modules, functions or procedures depending the framework and the language of your choice. Below, we give a description of sample classes that implement different stages of finding textually similar documents. You do not have to develop the exact same classes and data types as described below. Feel free to use data structures that suit you best.

A class Shingling that constructs k–shingles of a given length k (e.g., 10) from a given document, computes a hash value for each unique shingle, and represents the document in the form of an ordered set of its hashed k-shingles.
A class CompareSets that computes the Jaccard similarity of two sets of integers – two sets of hashed shingles.
A class MinHashing that builds a minHash signature (in the form of a vector or a set) of a given length n from a given set of integers (a set of hashed shingles).
A class CompareSignatures that estimates similarity of two integer vectors – minhash signatures – as a fraction of components, in which they agree.
(Optional task for extra 2 bonus) A class LSH that implements the LSH technique: given a collection of minhash signatures (integer vectors) and a similarity threshold t, the LSH class (using banding and hashing) finds candidate pairs of signatures agreeing on at least fraction t of their components.
To test and evaluate scalability (the execution time versus the size of input dataset) of your implementation, write a program that uses your classes to find similar documents in a corpus of 5-10 documents. Choose a similarity threshold s (e.g., 0,8) that states that two documents are similar if the Jaccard similarity of their shingle sets is at least s. 

## Method 

The classes will be implemented as Python functions.

In [2]:
import csv
import numpy as np
import re
import hashlib
import itertools
from collections import Counter
from pprint import pprint
import pandas as pd

In [3]:
# Produces shingles for a given text
def shingling(text, k = 2, trim = True):
    # possibly trim text
    s = re.sub('[\s+]', '', text) if trim else text

    # produce k-shingles and map shingles to shingle IDs using hash()
    shingles = {s[i:i + k] for i in range(len(s) - k + 1)}

    hashes = {hash(i) for i in shingles}
    return hashes

shingling('this is a test')

{-7203135641490933748,
 -6829267605805752621,
 -6789454011174268007,
 -5314113622642457652,
 1846648366803520841,
 3764021740757154572,
 5510198638609846826,
 7766216449880686841,
 7937168720576788187}

In [4]:
# Jaccard similarity for two sets
def compareSets(setA, setB):
    return len(setA.intersection(setB)) / len(setA.union(setB))

setA = shingling("how a nice day")
setB = shingling("how are you today")   
compareSets(setA,setB)

0.2777777777777778

In [5]:
# Creates the Minhash signature of length n from the shingle set
algorithms = [x for x in hashlib.algorithms_guaranteed if x not in ["shake_128", "shake_256"]] 
# they need a fixed-length argument(The shake_128() and shake_256() algorithms provide variable length digests with length_in_bits//2 up to 128 or 256 bits of security)

#hash a str into 8 digits using hash functions (hexdigest returns a HEX string representing the hash)
def hashWith(alg, i):
    return int(hashlib.new(alg,str(i).encode('UTF-8')).hexdigest(), 16)

def minHashing(shingleSet, n = 3):
    # throw error if not enough hash functions are available
    if (n > len(algorithms)):
        raise ValueError('The maximum number of hash functions available is {0}.'.format(len(algorithms)))

    # iterate over n hash functions and compute h_min(s) for the set.
    signature = [min(hashWith(alg, i) for i in shingleSet) for alg in algorithms[0:n]] 
    return signature

setA = shingling("how a nice day")
signature = minHashing(setA, 12)
signature

[219999983789596839455493963522010816285621917997982337728748363806,
 786029126074746716785257545659415459959373297086359467808737391442417822872533524452325780086190622096529910810679,
 96609120577086375298743731262274850804746671092,
 739447016452013557224137849509649045762804072342963900350319380697770104018983805860444208251749111574960191449942,
 544160981907356457025152255781454962744522988923040286164550473851952564720437732416913380372922424205801295132170464868319316527416494754097377706132217,
 257143993503011867438328337594533942392054373348456406114793506630835581506266203690509609270240802972794584883388028729430648287514865798529194676021374,
 5802550165512709507798071956094618899,
 2577106736418456394883600353311571602503213823261646351324035266051,
 7054476718490182837319462295996750530494891886941439033417043717052545500770,
 6158646141371861958515262949898497505037837514861641271148312992369511551549,
 881357243702096365236683348276524331547731974501408971918751953674

In [6]:
# Compare the signatures, the returned probability will approximate the jaccard similarity of the original shingle sets
def compareSignatures(signatureA, signatureB):
    if (len(signatureA) != len(signatureB)):
        raise ValueError('The signatures have different length({0}, {1} respectively) and should not be compared!', len(signatureA), len(signatureB))

    A = np.array(signatureA)
    B = np.array(signatureB)
    count = np.count_nonzero(A==B)
    # Important: Not jaccard similarity -> Probability instead (number_of_same/number_of_total)
    probability = count / len(signatureA)
    #print(A==B, count, probability)
    
    return probability

setA = shingling("what about you")
setB = shingling("how about you")
print("Set A", setA)
print("Set B", setB)
similarity = compareSets(setA, setB)
print(similarity)
signatureA = minHashing(setA, 12)
signatureB = minHashing(setB, 12)
print("Signature A", signatureA)
print("Signature B", signatureB)
compareSignatures(signatureA, signatureB)

Set A {-8957301752865267865, -5314113622642457652, -5028587291202181997, 2173489992638392465, -8118833194662279372, -3975535657283062222, 8627203540232068210, 8543914300129337426, -4663268017958072006, 4669754365558314877}
Set B {-8957301752865267865, -3900540923561869556, -7318321795911401232, 2173489992638392465, -3975535657283062222, -5028587291202181997, 8543914300129337426, -3458459600085340353, 4669754365558314877}
0.46153846153846156
Signature A [6520479235281335633732565341848356552193568697630085883070274962889, 17017145237257462087502244632711186712093825247196702372993562845503565907549234481412773509850810899809629959147, 23933812262506906556674202667015514511725239228, 3538233692629721733900047907022889781413722926232651745202099931117925028623720703380524972647088548190346087420102, 473230387848900049273495312219113081299628786841411392929781455532182393470792600866902715562691112119283644722463962866248410941841983935975585007979126, 2544740743276742494608420289651120936

0.5

The difference between the results of Jaccard similarity and Probability here is about 20%.

In [8]:
# Locality sensitive hashing , input parameter: # of bands to separate the signatures into
def lhs(signatures, similarity_threshold, nr_bands = 5, nr_buckets = 5):
    if (len(signatures) < 1 or not all(len(s) == len(signatures[0]) for s in signatures)):
        raise ValueError('The signatures need to have all the same length and be non empty.')

    # 1.) Iterate over signatures, cut in bands and hash each band into a bucket
    buckets = [set() for x in range(0, nr_buckets)]
    bands = np.linspace(0, len(signatures[0]), nr_bands).astype(int).tolist()

    for index, signature in enumerate(signatures):
        for i in range(0, nr_bands-1):
            band_start = bands[i]
            band_end = bands[i+1]
            # join band to be hashed "as one entity"
            band = "".join(str(x) for x in signature[band_start:band_end])
            bucket = hash(band) % nr_buckets
            # add the signature-set-join ("identifier") to the bucket
            #buckets[bucket].add("".join(str(x) for x in signature)) # store stringified signature
            buckets[bucket].add("index %s" % index) # replaced above line with a human readable string in order to validate results after
    
    # 2.) Use sets of buckets to determine candidate pairs based on threshold 
    relevant_buckets = [x for x in buckets if len(x) >= 2] # only check buckets with more than one signature       
    relevant_pairs = []
    for bucket in relevant_buckets:
        # get all relevant pairs from all buckets and append to a huge list
        pairs = [x for x in itertools.combinations(bucket, 2) if x[0] != x[1]]
        relevant_pairs += pairs

    count = Counter(relevant_pairs)
    # count the occourences of each pair in a list and see if the pairs similarity (based on same hashed buckets) crossed the threshold
    indices = [index for index, x in enumerate(count.values()) if (x/(nr_bands-1)) >= similarity_threshold] 
    
    # use the indices for the final candidate pairs
    candidate_pairs = [pair for index, pair in enumerate(count.keys()) if index in indices]

    return candidate_pairs

print(lhs([signatureA, signatureB], 0.3))
print(lhs([signatureA, signatureB], 0.8))

[('index 1', 'index 0')]
[]


#  Dataset and method implementation

We use the Eco-Hotel review dataset, which provides a corpus of 401 text-reviews as csv data and it contains one comment per line. Here we use defined functions on our dataset to find similar comments. Shingling size of 4 is chosen because commnents are short documents with a few lines of content (average character count is around 270).
Also, the function compare() is defined to help Jaccard similarity calculation. Threshold values are updated to keep a reasonable number of similar items.

In [9]:
import codecs
from pprint import pprint

# Import hotel review data
f = codecs.open('data.txt', encoding='utf-8')
dataSet = [line.strip() for line in f]

print('avg char count', sum(len(d) for d in dataSet)/len(dataSet))

# Executes given comparison function over combinations of elements in input array
def compare(fn, arr):
    return [(s[0][0], s[1][0], fn(s[0][1], s[1][1])) for s in itertools.combinations(arr, 2)]

# jaccard similarity with shinglings
shinglings = [(i+1, shingling(t, k=4)) for i, t in enumerate(dataSet)] # value i+1 is the line number and it is stored to evalute results
similarities = compare(compareSets, shinglings)

similarities = [s for s in similarities if s[2] > 0.3]
pprint(similarities)

# jaccard similarity with minHashing
signatures = [(i, minHashing(s, n=12)) for i, s in shinglings]
similarities = compare(compareSignatures, signatures)

similarities = [s for s in similarities if s[2] > 0.4]
pprint(similarities)

# local sensitive hashing
pprint(lhs([s for i, s in signatures], 0.5, nr_bands = 4, nr_buckets=200)) # index i represents the comment with line number i+1

avg char count 268.6708229426434
[(73, 74, 1.0),
 (90, 186, 0.4090909090909091),
 (90, 194, 0.3235294117647059),
 (102, 140, 0.3492063492063492),
 (153, 154, 1.0),
 (178, 241, 0.375),
 (178, 395, 0.3333333333333333),
 (235, 347, 0.34146341463414637),
 (239, 258, 0.34285714285714286),
 (239, 347, 0.3333333333333333),
 (241, 395, 0.30303030303030304),
 (258, 320, 0.34285714285714286),
 (258, 347, 0.34375),
 (258, 351, 0.3055555555555556),
 (258, 366, 0.3783783783783784),
 (320, 328, 0.34615384615384615),
 (320, 330, 0.3055555555555556),
 (320, 347, 0.3333333333333333),
 (320, 377, 0.3055555555555556),
 (320, 382, 0.32075471698113206),
 (330, 377, 0.3142857142857143)]
[(6, 102, 0.4166666666666667),
 (6, 124, 0.5),
 (6, 130, 0.4166666666666667),
 (10, 88, 0.4166666666666667),
 (10, 124, 0.4166666666666667),
 (10, 194, 0.4166666666666667),
 (10, 364, 0.4166666666666667),
 (11, 97, 0.4166666666666667),
 (11, 151, 0.4166666666666667),
 (11, 239, 0.4166666666666667),
 (11, 258, 0.4166666666666

We can see that the results of Jaccard similarity with shingling and min hashing are similar. While result of LSH is slightly different due to low number of bands. There are 12 hashing algorithms used for Min Hashing, we use highest vector size of 12. They are split into 4 bands with 3 rows per band.

We can see that pairs with Jaccard similarity of 1.0 are identical. If we check the review dataset we can see that comments on line 73 and 153 are the same. 

We can also check Pair(90, 186) in the reviews file, which is found both in shingling and min hashing method, and we can see that comments are very similar.

Line 90: "We will come back"

Line 186: "I will come back, thank you."